# Import stuff

In [ ]:
import numpy as np
from random import uniform
from astropy.io import fits

# Load the data

In [ ]:
f = fits.open('/home/sean/Downloads/LOFAR_HBA_T1_DR1_catalog_v0.9.srl.fixed.fits') # LoTSS

h = f[1]
ra        = h.data['RA'       ] # degrees
dec       = h.data['DEC'      ] # degrees
ra_error  = h.data['E_RA_tot' ] # arcseconds
dec_error = h.data['E_DEC_tot'] # arcseconds
f.close()

# Calculate LoTSS position ranges

In [ ]:
ra_low, ra_high, dec_low, dec_high = [], [], [], []

for i in range(len(ra)):
    ra_low.append(ra[i]    - (ra_error[i]  / (60 * 60)))
    ra_high.append(ra[i]   + (ra_error[i]  / (60 * 60)))
    dec_low.append(dec[i]  - (dec_error[i] / (60 * 60)))
    dec_high.append(dec[i] + (dec_error[i] / (60 * 60)))
    
ra_min  = min(ra_low)
ra_max  = max(ra_high)
dec_min = min(dec_low)
dec_max = max(dec_high)

# Generate source positions to test

In [ ]:
ra_test, dec_test = [], []

for i in range(10000):
    ra_test.append(uniform(ra_min, ra_max))
    dec_test.append(uniform(dec_min, dec_max))
    
test_error = 0.1 / (60 * 60) # degrees

# Check for spurious cross-matches

In [ ]:
match = 0

for i in range(len(ra_test)): # iterate through test points
    for j in range(len(ra_low)): # iterate through catalogue
        if ra_test[i] + test_error > ra_low[j] and ra_test[i] - test_error < ra_high[j]: # check if RA overlaps
            if dec_test[i] + test_error > dec_low[j] and dec_test[i] - test_error < dec_high[j]: # check if declination overlaps
                match += 1
                print(match, ra_test[i], ra_low[j], ra_high[j], dec_test[i], dec_low[j], dec_high[j])

In [ ]:
fudge = 1.2
percent = (match / len(ra_test)) * 100
print(str(match) + '/' + str(len(ra_test)) + ' (' + str(percent) + '%; ' + str(percent * fudge) + '% with fudge)')
print('Average LoTSS positional uncertainty = ' + str(round((np.mean(ra_error) + np.mean(dec_error)) / 2, 4)) + '"')